<a href="https://colab.research.google.com/github/JayRothenberger/CarCrashProject/blob/master/CarCrashProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Car Crashes in Virginia**
**CS 4774: Machine Learning for VA (ML4VA)**

Jay Rothenberger (jr2fh); Sindhu Ranga (sgr7vc); Sarah Zhou (syz3bv)
****
**Premise**:

**Goal**:


NOTE: Data set is 800k+ rows! Use a small sample size to test

# 1. Setup

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
# any number will do, as long as it is used consistently
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# 2. Get Dataset

In [ ]:
# load dataset
import os # in order to run file I/O operation 
from six.moves import urllib # support URL download

DOWNLOAD_ROOT = "https://media.githubusercontent.com/media/sranga12/mlprojectcars/master/"
DATA_PATH = os.path.join("datasets", "crashes")
DATA_URL = DOWNLOAD_ROOT + "Virginia_Crashes_102919_final.csv"

def fetch_data(data_url=DATA_URL, data_path=DATA_PATH):
    """Fetch data from a remote URL to Colab file system"""
    if not os.path.isdir(data_path):
        os.makedirs(data_path)
    csv_path = os.path.join(data_path, "Virginia_Crashes_102919_final.csv")
    urllib.request.urlretrieve(data_url, csv_path)

In [ ]:
fetch_data()

In [ ]:
import pandas as pd

def load_data(data_path = DATA_PATH):
    """Load Data into Workspace from a CSV"""
    csv_path = os.path.join(data_path,"Virginia_Crashes_102919_final.csv")
    columns = ["X", "Y", "VDOT_District", "Rte_Nm",
                                                "Crash_Year", "Crash_Mo", "Crash_Day", 
                                                "Crash_Military_Tm", "Collision_Type", 
                                                "Crash_Severity", "Persons_Killed", 
                                                "Light_Condition", "Weather_Condition", 
                                                "Roadway_Surface_Cond", "VSP_Used", "Rns_Mp", 
                                                "Driverage_average", "Drivergen_average"]
    df = pd.read_csv(csv_path)
    return df.drop(df.index[0])

In [ ]:
crashes = load_data()
crashes_list = []

print(crashes.head(10))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


            X          Y  ... Unnamed: 18 Unnamed: 19
1  -80.898183  36.947187  ...         NaN         NaN
2  -80.578353  37.090327  ...         NaN         NaN
3  -78.926593  38.358578  ...         NaN         NaN
4  -77.610592  39.140818  ...         NaN         NaN
5  -77.466652  38.367778  ...         NaN         NaN
6  -76.684652  37.305017  ...         NaN         NaN
7  -79.827623  37.277897  ...         NaN         NaN
8  -77.104892  37.785908  ...         NaN         NaN
9  -77.606532  39.139458  ...         NaN         NaN
10 -77.564362  38.712618  ...         NaN         NaN

[10 rows x 20 columns]


In [ ]:
# Provide a quick summary of the data including name of features, count, and type
crashes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 817474 entries, 1 to 817474
Data columns (total 20 columns):
X                       815986 non-null float64
Y                       815986 non-null float64
VDOT_District           817474 non-null object
Rte_Nm                  817242 non-null object
Crash_Dt                817474 non-null object
Crash_Military_Tm       817474 non-null int64
Collision_Type          817474 non-null object
Crash_Severity          817474 non-null object
Persons_Killed          817474 non-null int64
Light_Condition         817474 non-null object
Weather_Condition       817474 non-null object
Roadway_Surface_Cond    817474 non-null object
VSP_Used                817474 non-null int64
Rns_Mp                  771815 non-null float64
Driverage_average       817474 non-null object
Drivergen_average       817474 non-null float64
Unnamed: 16             0 non-null float64
Unnamed: 17             0 non-null float64
Unnamed: 18             0 non-null float64
Unnamed

# 3. Data Cleaning

In [ ]:
crashes.iloc[23572]

X                                                     -77.3281
Y                                                      38.6398
VDOT_District                              9.Northern Virginia
Rte_Nm                                        R-VA076SC00784EB
Crash_Dt                              2013-01-10T00:00:00.000Z
Crash_Military_Tm                                         1625
Collision_Type                      9. Fixed Object - Off Road
Crash_Severity                                 A.Severe Injury
Persons_Killed                                               0
Light_Condition                                        3. Dusk
Weather_Condition       1. No Adverse Condition (Clear/Cloudy)
Roadway_Surface_Cond                                    1. Dry
VSP_Used                                                     7
Rns_Mp                                                 4.86926
Driverage_average                                           25
Drivergen_average                                      

In [ ]:
del crashes['Rte_Nm']
del crashes['Unnamed: 16']
del crashes['Unnamed: 17']
del crashes['Unnamed: 18']
del crashes['Unnamed: 19']
del crashes['Crash_Dt']
divzeros = crashes.loc[crashes['Driverage_average'] == '#DIV/0!']
crashes = crashes.drop(divzeros.index)
incomplete_rows =  crashes[crashes.isnull().any(axis=1)]
crashes = crashes.drop(incomplete_rows.index)

for col in crashes:
  NA = crashes.loc[crashes[col] == 'Not Applicable']
  crashes = crashes.drop(NA.index)

notprovided = crashes.loc[crashes['Collision_Type'] == 'Not Provided']
crashes = crashes.drop(notprovided.index)
crashes = crashes.dropna()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [ ]:
crashes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 759465 entries, 1 to 817474
Data columns (total 14 columns):
X                       759465 non-null float64
Y                       759465 non-null float64
VDOT_District           759465 non-null object
Crash_Military_Tm       759465 non-null int64
Collision_Type          759465 non-null object
Crash_Severity          759465 non-null object
Persons_Killed          759465 non-null int64
Light_Condition         759465 non-null object
Weather_Condition       759465 non-null object
Roadway_Surface_Cond    759465 non-null object
VSP_Used                759465 non-null int64
Rns_Mp                  759465 non-null float64
Driverage_average       759465 non-null object
Drivergen_average       759465 non-null float64
dtypes: float64(4), int64(3), object(7)
memory usage: 86.9+ MB


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# split dataset
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(crashes, test_size=0.05, train_size = 0.3)

X_train = train_set.drop("Crash_Severity", axis=1); # drop labels for training set
y_train = train_set["Crash_Severity"].copy();
X_test = test_set.drop("Crash_Severity", axis=1);
y_test = test_set["Crash_Severity"].copy();

y_train = np.array([y_train]).T
y_test = np.array([y_test]).T


num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()), 
]) 

num_attribs = list(crashes.drop(["VDOT_District", "Collision_Type", 
                                 "Light_Condition", "Weather_Condition", 
                                 "Roadway_Surface_Cond", "Crash_Severity"], axis=1))
cat_attribs = ["VDOT_District", "Collision_Type", "Light_Condition", 
               "Weather_Condition", "Roadway_Surface_Cond"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("ohe", OneHotEncoder(), cat_attribs), 
    ])

X_train = full_pipeline.fit_transform(X_train)
X_test = full_pipeline.transform(X_test)

X_valid = X_train[:45000]
X_train = X_train[45000:]

y_valid = y_train[:45000]
y_train = y_train[45000:]

# are dimensions correct?
print(np.shape(X_test)) # x
print(np.shape(y_train)) # x1
print(np.shape(y_valid))

(37974, 61)
(182838, 1)
(45000, 1)


# 5. Testing Different Models

## SVM

In [ ]:
# test different models for data (and explain reasoning in text)
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score as accuracy

In [ ]:
print(X_test.shape, y_test.shape)

(37974, 61) (37974, 1)


In [ ]:
# Linear OvR
linear_ovr_svm = SVC(kernel="linear", C=1, gamma="scale", decision_function_shape="ovr")
linear_ovr_svm.fit(X_train, y_train)
y_pred=linear_ovr_svm.predict(X_test)
cv = accuracy(y_pred, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
print(cv)

0.6604344963791968


In [ ]:
# Linear OvO
linear_ovo_svm = SVC(kernel="linear", C=1, gamma="scale", decision_function_shape="ovo")
linear_ovo_svm.fit(X_train, y_train)
y_pred=linear_ovo_svm.predict(X_test)
print(accuracy(y_pred, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6608294930875576


In [ ]:
# RBF Kernel
linear_ovo_svm = SVC(kernel="rbf", C=1, gamma="scale", decision_function_shape="ovo")
linear_ovo_svm.fit(X_train, y_train)
y_pred=linear_ovo_svm.predict(X_test)
print(accuracy(y_pred, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6603028308097433


In [ ]:
# RBF Kernel
rbf_ovr_svm = SVC(kernel="rbf", C=1, gamma="scale", decision_function_shape="ovr")
rbf_ovr_svm.fit(X_train, y_train)
y_pred=rbf_ovr_svm.predict(X_test)
print(accuracy(y_pred, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6603028308097433


## Neural Network sans Ensemble

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
# INSERT THE LINE BELOW TO SELECT TENSORFLOW 2.0
%tensorflow_version 2.x
# TensorFlow ≥2.0-preview is required
import tensorflow as tf
print(tf.__version__)
#assert tf.__version__ >= "2.0"
from tensorflow import keras
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
model = keras.models.Sequential([
keras.layers.Dense(X_train.shape[1],input_dim=X_train.shape[1], activation='relu'),
keras.layers.Dense(200, activation="relu"),
keras.layers.Dense(150, activation="relu"),
keras.layers.Dense(120, activation="relu"),
keras.layers.Dense(81, activation="softmax"),
])

#.6615

2.0.0


In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=[keras.metrics.sparse_categorical_accuracy])

# Encoding the data to be entirely numerical for the NN

In [ ]:
def encode_y(arr):
  for i in range(len(arr)):
    arr[i][0] = ord(arr[i][0][0])
  #z = max(arr) + 1
  z=1
  for i in range(len(arr)):
    arr[i] /= z
  return arr

y_train = encode_y(y_train)
y_valid = encode_y(y_valid)
y_test = encode_y(y_test)

In [ ]:
X_train.todense()
X_valid.todense()
X_test.todense()

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))
model.evaluate(X_test, y_test)

Train on 182838 samples, validate on 45000 samples
Epoch 1/10
182838/182838 [==============================] - 15s 82us/sample - loss: 1.0077 - sparse_categorical_accuracy: 0.6542 - val_loss: 0.9311 - val_sparse_categorical_accuracy: 0.6591
Epoch 2/10
182838/182838 [==============================] - 15s 81us/sample - loss: 0.9303 - sparse_categorical_accuracy: 0.6577 - val_loss: 0.9187 - val_sparse_categorical_accuracy: 0.6625
Epoch 3/10
182838/182838 [==============================] - 16s 85us/sample - loss: 0.9210 - sparse_categorical_accuracy: 0.6607 - val_loss: 0.9314 - val_sparse_categorical_accuracy: 0.6619
Epoch 4/10
182838/182838 [==============================] - 16s 88us/sample - loss: 0.9169 - sparse_categorical_accuracy: 0.6613 - val_loss: 0.9145 - val_sparse_categorical_accuracy: 0.6630
Epoch 5/10
182838/182838 [==============================] - 18s 98us/sample - loss: 0.9145 - sparse_categorical_accuracy: 0.6613 - val_loss: 0.9150 - val_sparse_categorical_accuracy: 0.6631

[0.9156903901823187, 0.6602149]

## A Basic Shallow Sequential Neural Network

In [ ]:
X_train.todense()
X_valid.todense()
X_test.todense()

model = keras.models.Sequential([
keras.layers.Dense(200,input_dim=X_train.shape[1], activation='relu'),
keras.layers.Dense(200, activation="relu"),
keras.layers.Dense(150, activation="relu"),
keras.layers.Dense(120, activation="relu"),
keras.layers.Dense(120, activation="relu"),
keras.layers.Dense(81, activation="softmax"),
])
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=[keras.metrics.sparse_categorical_accuracy])

history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

model.evaluate(X_test, y_test)
#0.6659

Train on 182839 samples, validate on 45000 samples
Epoch 1/10
182839/182839 [==============================] - 19s 103us/sample - loss: 0.9990 - sparse_categorical_accuracy: 0.6550 - val_loss: 0.9384 - val_sparse_categorical_accuracy: 0.6586
Epoch 2/10
182839/182839 [==============================] - 19s 102us/sample - loss: 0.9310 - sparse_categorical_accuracy: 0.6580 - val_loss: 0.9276 - val_sparse_categorical_accuracy: 0.6615
Epoch 3/10
182839/182839 [==============================] - 18s 99us/sample - loss: 0.9221 - sparse_categorical_accuracy: 0.6606 - val_loss: 0.9191 - val_sparse_categorical_accuracy: 0.6621
Epoch 4/10
182839/182839 [==============================] - 18s 99us/sample - loss: 0.9173 - sparse_categorical_accuracy: 0.6611 - val_loss: 0.9142 - val_sparse_categorical_accuracy: 0.6624
Epoch 5/10
182839/182839 [==============================] - 18s 98us/sample - loss: 0.9149 - sparse_categorical_accuracy: 0.6614 - val_loss: 0.9259 - val_sparse_categorical_accuracy: 0.66

[0.9147406370774035, 0.6606099]

## Building the Ensemble of Shallow Networks

In [ ]:
from sklearn.metrics import accuracy_score
from numpy import argmax
#ensemble of the models:
def new_trained_model():
  train_set, test_set = train_test_split(crashes, test_size=0.15, train_size = 0.3)

  X_train = train_set.drop("Crash_Severity", axis=1); # drop labels for training set
  y_train = train_set["Crash_Severity"].copy();

  y_train = np.array([y_train]).T

  num_pipeline = Pipeline([
      ('std_scaler', StandardScaler()), 
  ]) 

  num_attribs = list(crashes.drop(["VDOT_District", "Collision_Type", 
                                  "Light_Condition", "Weather_Condition", 
                                  "Roadway_Surface_Cond", "Crash_Severity"], axis=1))
  cat_attribs = ["VDOT_District", "Collision_Type", "Light_Condition", 
                "Weather_Condition", "Roadway_Surface_Cond"]

  full_pipeline = ColumnTransformer([
          ("num", num_pipeline, num_attribs),
          ("ohe", OneHotEncoder(), cat_attribs), 
      ])

  X_train = full_pipeline.fit_transform(X_train)

  X_valid = X_train[:45000]
  X_train = X_train[45000:]

  y_valid = y_train[:45000]
  y_train = y_train[45000:]
  #now we have new data, we need to encode it
  y_train = encode_y(y_train)
  y_valid = encode_y(y_valid)
  #here we have encoded it, now we need to create a new model

  X_train.todense()
  X_valid.todense()

  model = keras.models.Sequential([
  keras.layers.Dense(200,input_dim=61, activation='relu'),
  keras.layers.Dense(200, activation="relu"),
  keras.layers.Dense(150, activation="relu"),
  keras.layers.Dense(120, activation="relu"),
  keras.layers.Dense(81, activation="softmax"),
  ])
  model.compile(loss=keras.losses.sparse_categorical_crossentropy,
                optimizer=keras.optimizers.SGD(),
                metrics=[keras.metrics.sparse_categorical_accuracy])

  trained = model.fit(X_train, y_train, epochs=3,
                      validation_data=(X_valid, y_valid))
  return model

senators = []
for i in range(7):
  senators.append(new_trained_model())


Train on 182838 samples, validate on 45000 samples
Epoch 1/3
182838/182838 [==============================] - 18s 98us/sample - loss: 0.9888 - sparse_categorical_accuracy: 0.6543 - val_loss: 0.9327 - val_sparse_categorical_accuracy: 0.6597
Epoch 2/3
182838/182838 [==============================] - 18s 97us/sample - loss: 0.9257 - sparse_categorical_accuracy: 0.6594 - val_loss: 0.9297 - val_sparse_categorical_accuracy: 0.6602
Epoch 3/3
182838/182838 [==============================] - 19s 107us/sample - loss: 0.9197 - sparse_categorical_accuracy: 0.6605 - val_loss: 0.9251 - val_sparse_categorical_accuracy: 0.6621
Train on 182838 samples, validate on 45000 samples
Epoch 1/3
182838/182838 [==============================] - 21s 112us/sample - loss: 0.9850 - sparse_categorical_accuracy: 0.6555 - val_loss: 0.9441 - val_sparse_categorical_accuracy: 0.6535
Epoch 2/3
182838/182838 [==============================] - 21s 112us/sample - loss: 0.9249 - sparse_categorical_accuracy: 0.6596 - val_loss:

## Implementing the Basic Voting Strategy

In [ ]:
from numpy import array
import statistics
def vote(models, X_test):
	y = []

	y_pred = [model.predict_classes(X_test) for model in models]
	y_pred = array(y_pred).T

	print(y_pred)

	for i in range(len(y_pred)):
		try:
			y.append([statistics.mode(y_pred[i])])
		except:
			y.append([statistics.median(y_pred[i])])
	
	print(array(y))

	return array(y)

def evaluate_senators(members, X_test, y_test):
	same = 0
	y_pred = vote(members, X_test)
	for i in range(len(y_pred)):
		if y_pred[i] == y_test[i]:
			same+=1
	return same/len(y_test)

print(evaluate_senators(senators, X_test, y_test))
#.65676

[[80 80 80 ... 80 80 80]
 [80 80 80 ... 80 80 80]
 [80 80 66 ... 80 80 80]
 ...
 [80 80 80 ... 80 80 80]
 [80 80 80 ... 80 80 80]
 [80 80 80 ... 80 80 80]]
[[80]
 [80]
 [80]
 ...
 [80]
 [80]
 [80]]
0.6608833430624257


In [ ]:
train_set, test_set = train_test_split(crashes, test_size=0.3, train_size = 0.7)

X_train = train_set.drop("Crash_Severity", axis=1); # drop labels for training set
y_train = train_set["Crash_Severity"].copy();
X_test = test_set.drop("Crash_Severity", axis=1);
y_test = test_set["Crash_Severity"].copy();

y_train = np.array([y_train]).T
y_test = np.array([y_test]).T


num_pipeline = Pipeline([
    ('std_scaler', StandardScaler()), 
]) 

num_attribs = list(crashes.drop(["VDOT_District", "Collision_Type", 
                                 "Light_Condition", "Weather_Condition", 
                                 "Roadway_Surface_Cond", "Crash_Severity"], axis=1))
cat_attribs = ["VDOT_District", "Collision_Type", "Light_Condition", 
               "Weather_Condition", "Roadway_Surface_Cond"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("ohe", OneHotEncoder(), cat_attribs), 
    ])

X_train = full_pipeline.fit_transform(X_train)
X_test = full_pipeline.transform(X_test)

X_valid = X_train[:45000]
X_train = X_train[45000:]

y_valid = y_train[:45000]
y_train = y_train[45000:]

y_train = encode_y(y_train)
y_valid = encode_y(y_valid)
y_test = encode_y(y_test)
#here we have encoded it, now we need to create a new model

X_train.todense()
X_valid.todense()
X_test.todense()

model = keras.models.Sequential([
keras.layers.Dense(200,input_dim=61, activation='elu'),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(200, activation="elu"),
keras.layers.Dense(150, activation="elu"),
keras.layers.Dense(120, activation="elu"),
keras.layers.Dense(100, activation="softmax"),
])
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=[keras.metrics.sparse_categorical_accuracy])

trained = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))
#6606

Train on 486624 samples, validate on 45000 samples
Epoch 1/10
486624/486624 [==============================] - 124s 255us/sample - loss: 0.9371 - sparse_categorical_accuracy: 0.6592 - val_loss: 0.9128 - val_sparse_categorical_accuracy: 0.6628
Epoch 2/10
486624/486624 [==============================] - 123s 253us/sample - loss: 0.9171 - sparse_categorical_accuracy: 0.6607 - val_loss: 0.9125 - val_sparse_categorical_accuracy: 0.6620
Epoch 3/10
486624/486624 [==============================] - 122s 250us/sample - loss: 0.9126 - sparse_categorical_accuracy: 0.6609 - val_loss: 0.9059 - val_sparse_categorical_accuracy: 0.6627
Epoch 4/10
486624/486624 [==============================] - 136s 279us/sample - loss: 0.9074 - sparse_categorical_accuracy: 0.6609 - val_loss: 0.8999 - val_sparse_categorical_accuracy: 0.6628
Epoch 5/10
486624/486624 [==============================] - 129s 265us/sample - loss: 0.9041 - sparse_categorical_accuracy: 0.6610 - val_loss: 0.8981 - val_sparse_categorical_accura

## A Deep 103 Layer Network

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(200,input_dim=61, activation='elu'))
model.add(keras.layers.Dense(200, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(100):
    model.add(keras.layers.Dense(200, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(81, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])


In [ ]:
history = model.fit(X_train, y_train, epochs=3,
                    validation_data=(X_valid, y_valid))

Train on 486624 samples, validate on 45000 samples
Epoch 1/3
486624/486624 [==============================] - 731s 2ms/sample - loss: 0.9449 - accuracy: 0.6575 - val_loss: 0.9244 - val_accuracy: 0.6610
Epoch 2/3
486624/486624 [==============================] - 677s 1ms/sample - loss: 0.9189 - accuracy: 0.6609 - val_loss: 0.9080 - val_accuracy: 0.6629
Epoch 3/3
486624/486624 [==============================] - 715s 1ms/sample - loss: 0.9108 - accuracy: 0.6609 - val_loss: 0.9129 - val_accuracy: 0.6628
